<a href="https://colab.research.google.com/github/RaminParker/Text-Classification-with-Python/blob/master/Text_classification_with_balancing_20newsgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is part 2 of the news article classification. Check out [part 1](https://github.com/RaminParker/Text-Classification-with-Python/blob/master/Text_Classification_using_scikit_learn_(1).ipynb)

In [0]:
import os
import string
import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.metrics import classification_report_imbalanced
from imblearn.under_sampling import RandomUnderSampler

from sklearn.datasets import fetch_20newsgroups
from collections import Counter

Define categories which you are interested in:

In [0]:
# categories = ['alt.atheism',
#  'comp.graphics',
#  'comp.os.ms-windows.misc',
#  'comp.sys.ibm.pc.hardware',
#  'comp.sys.mac.hardware',
#  'comp.windows.x',
#  'misc.forsale',
#  'rec.autos',
#  'rec.motorcycles',
#  'rec.sport.baseball',
#  'rec.sport.hockey',
#  'sci.crypt',
#  'sci.electronics',
#  'sci.med',
#  'sci.space',
#  'soc.religion.christian',
#  'talk.politics.guns',
#  'talk.politics.mideast',
#  'talk.politics.misc',
#  'talk.religion.misc']

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [0]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

Look at the data:

In [39]:
print(newsgroups_train.data[0][:80])
print(newsgroups_train.data[1][:80])
print(newsgroups_train.data[2][:80])

From: rych@festival.ed.ac.uk (R Hawkes)
Subject: 3DS: Where did all the texture 
Subject: Re: Biblical Backing of Koresh's 3-02 Tape (Cites enclosed)
From: kmcva
From: Mark.Perew@p201.f208.n103.z1.fidonet.org
Subject: Re: Comet in Temporary O


Split data:

In [0]:
X_train = newsgroups_train.data
X_test = newsgroups_test.data

y_train = newsgroups_train.target
y_test = newsgroups_test.target

# The usual scikit-learn pipeline

In [0]:
pipe = make_pipeline(TfidfVectorizer(), MultinomialNB()) # build pipeline

In [0]:
pipe.fit(X_train, y_train) # fit to data

In [0]:
y_pred = pipe.predict(X_test)

In [24]:
print(classification_report_imbalanced(y_test, y_pred)) # classification report 

                   pre       rec       spe        f1       geo       iba       sup

          0       0.67      0.94      0.86      0.79      0.90      0.82       319
          1       0.96      0.92      0.99      0.94      0.95      0.90       389
          2       0.87      0.98      0.94      0.92      0.96      0.92       394
          3       0.97      0.36      1.00      0.52      0.60      0.33       251

avg / total       0.87      0.84      0.94      0.82      0.88      0.78      1353



-- > The recall of the class #3 is low mainly due to the class imbalanced.

# Balancing the class before classification

Improve prediction of the class #3: apply a balancing before to train the naive bayes classifier. 

--> **RandomUnderSampler** to equalize the number of samples in all the classes before the training.

In [0]:
pipe = make_pipeline_imb(TfidfVectorizer(), RandomUnderSampler(), MultinomialNB())  # build pipeline

In [45]:
pipe.fit(X_train, y_train) # fit to data

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('randomundersampler',
                 RandomUnderSampler(random_state=None, ratio=None,
                                    replacement=False, 

In [0]:
y_pred = pipe.predict(X_test) # classification report 

In [47]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.72      0.89      0.89      0.80      0.89      0.79       319
          1       0.98      0.85      0.99      0.91      0.92      0.83       389
          2       0.93      0.91      0.97      0.92      0.94      0.88       394
          3       0.78      0.74      0.95      0.76      0.84      0.69       251

avg / total       0.87      0.86      0.96      0.86      0.90      0.81      1353



Although the results are almost identical, it can be seen that the resampling allowed to correct the poor recall of the class #3 at the cost of reducing the other metrics for the other classes. However, the overall results are slightly better.